# MLflow を使用して Git ベースのアプリケーションのバージョンを追跡する

https://docs.databricks.com/aws/ja/mlflow3/genai/prompt-version-mgmt/version-tracking/track-application-versions-with-mlflow

**LoggedModel** は、Experiment 内で AI モデル、エージェント、または GenAI アプリケーションを表すファーストクラスのエンティティです。開発ライフサイクル全体を通じて、モデルアーティファクト、実行トレース、評価指標、メタデータの統合トラッキングを提供します。

## LoggedModel の主な機能
LoggedModel は次のものを接続する中心的なハブとして機能します:

- **モデルアーティファクト** と構成パラメータ
- 開発および本番環境からの **実行トレース**
- **評価指標** とパフォーマンス評価
- **バージョン履歴** とデプロイ追跡


## LoggedModel の利点
**統合トラッキング:** すべてのモデル関連アーティファクト、トレース、メトリクスが単一のエンティティの下に整理され、モデルの挙動とパフォーマンスを完全に可視化できます。

**自動トレースリンク:** アクティブモデルパターンを使用すると、すべての実行トレースが自動的に LoggedModel に関連付けられ、手動での追跡作業が不要になります。

**バージョン管理:** LoggedModel は、異なるモデル反復間での体系的なバージョン管理と比較をサポートし、データ駆動型のモデル選択を可能にします。

**評価統合:** 評価指標や結果が直接 LoggedModel にリンクされ、包括的なパフォーマンス評価が可能です。

In [0]:
%pip install --upgrade "mlflow[databricks]>=3.1.0" openai

%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import mlflow
from openai import OpenAI

# MLflowの自動ロギングを有効にして、アプリケーションにトレースを追加
mlflow.openai.autolog()

# トレースデコレータを使用してアプリケーションのエントリポイントをキャプチャ
@mlflow.trace
def my_app(model_input: str):
    # Notebook実行の資格情報を利用してDatabricks Foundation Model APIへのCredentialを取得
    mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
    client = OpenAI(
        api_key=mlflow_creds.token, base_url=f"{mlflow_creds.host}/serving-endpoints"
    )

    # この呼び出しは`mlflow.openai.autolog()`によって自動的に計測されます
    response = client.chat.completions.create(
        model="databricks-meta-llama-3-3-70b-instruct",
        messages=[
            {
                "role": "system",
                "content": "あなたは役に立つアシスタントです。",
            },
            {
                "role": "user",
                "content": model_input,
            },
        ],
    )
    return response.choices[0].message.content

# テスト実行
result = my_app(model_input="MLflowとは何ですか？")
print(result)

MLflowは、機械学習モデルの管理、開発、デプロイを支援するオープンソースのプラットフォームです。


Trace(trace_id=tr-fe1ae80d0414f811a8f3144616f45931)

In [0]:
model_info = mlflow.pyfunc.log_model(
    name="my_app",
    python_model=my_app,
    pip_requirements=["openai"],
)

🔗 View Logged Model at: https://dbc-f1d46cb2-e7aa.cloud.databricks.com/ml/experiments/5cdd0258ddc24c8bbe19dfe08a33daee/models/m-ab3ffe66db86421e801934ce33a10a7c?o=1765512908890676
/local_disk0/.ephemeral_nfs/envs/pythonEnv-6e58b23d-d637-4ea9-a8a5-cdd09e2846da/lib/python3.11/site-packages/mlflow/pyfunc/utils/data_validation.py:47: UserWarning: Type hint used in the model's predict function is not supported for MLflow's schema validation. Type hints must be wrapped in list[...] because MLflow assumes the predict method to take multiple input instances. Specify your type hint as `list[str]` for a valid signature. Remove the type hint to disable this warning. To enable validation for the input data, specify input example or model signature when logging the model. 
  func_info = _get_func_info_if_type_hint_supported(func)
/local_disk0/.ephemeral_nfs/envs/pythonEnv-6e58b23d-d637-4ea9-a8a5-cdd09e2846da/lib/python3.11/site-packages/mlflow/pyfunc/__init__.py:3191: UserWarning: Failed to infer s

In [0]:
model = mlflow.pyfunc.load_model(model_info.model_uri)
prediction = model.predict("MLflowとは何ですか？")
print(prediction)


MLflowは、機械学習の開発ライフサイクルを管理するために使用されるオープンソースのソフトウェアです。

MLflowは、以下のような機能を提供します：

1.  **実験名管理**：実験名（実験）は、機械学習モデルのトレーニングやテストの実行を管理する単位です。MLflowでは、実験名を登録して管理できます。
2.  **パラメータ管理**：機械学習モデルをトレーニングする際に使用するパラメータ（ハイパーパラメータ）を管理できます。
3.  **メトリック管理**：実験名の結果として出力されるメトリック（精度、損失関数など）を管理できます。
4.  **モデル管理**：トレーニングされた機械学習モデルを管理できます。
5.  **モデルサービング**：トレーニングされたモデルをWeb APIとして公開し、予測を要求できるようにします。

MLflowは、機械学習の開発プロセスを効率化し、再現性を高め、チームでの協業を容易にすることを目指しています。


Trace(trace_id=tr-5aba90fc81e8dba76742cad0e2b41dba)

## バージョン管理

In [0]:
import mlflow
from openai import OpenAI

mlflow.openai.autolog()

@mlflow.trace
def my_app_revised(model_input: str):
    # Notebook実行の資格情報を利用してDatabricks Foundation Model APIへのCredentialを取得
    mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
    client = OpenAI(
        api_key=mlflow_creds.token, base_url=f"{mlflow_creds.host}/serving-endpoints"
    )

    # 利用するモデルをLlama3.3 70BからLlama4 に変更する
    response = client.chat.completions.create(
        model="databricks-llama-4-maverick",
        messages=[
            {
                "role": "system",
                "content": "あなたは役に立つアシスタントです。",
            },
            {
                "role": "user",
                "content": model_input,
            },
        ],
    )
    return response.choices[0].message.content

# テスト実行
result = my_app_revised(model_input="MLflowとは何ですか？")
print(result)

MLflowは、機械学習のライフサイクルを管理するためのオープンソースプラットフォームです。MLflowは、データの前処理、モデルのトレーニング、ハイパーパラメータのチューニング、モデルのデプロイなど、機械学習プロジェクトのさまざまな段階を管理するためのツールを提供します。

MLflowの主な機能は以下のとおりです。

1. **実験の管理**: MLflowは、機械学習実験を管理するためのツールを提供します。実験のパラメータ、メトリクス、成果物を追跡し、比較することができます。
2. **モデルの管理**: MLflowは、機械学習モデルを管理するためのツールを提供します。モデルを保存、読み込み、デプロイすることができます。
3. **再現性の確保**: MLflowは、機械学習実験の再現性を確保するためのツールを提供します。実験の環境、パラメータ、データなどを記録し、再現することができます。

MLflowを使用することで、以下のようなメリットがあります。

* 機械学習プロジェクトの管理が容易になる
* 実験の再現性が確保される
* モデルのデプロイが容易になる
* チームでの共同作業が容易になる

MLflowは、Python、R、Javaなどのさまざまなプログラミング言語に対応しており、さまざまな機械学習ライブラリ（TensorFlow、PyTorch、Scikit-learnなど）と統合することができます。


Trace(trace_id=tr-bb9841ce93df91e6e2528bf791420904)

In [0]:
active_model_info = mlflow.set_active_model(name="my_app_dev")

2025/06/28 09:13:11 INFO mlflow.tracking.fluent: LoggedModel with name 'my_app_dev' does not exist, creating one...
2025/06/28 09:13:12 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-8979bb49155245f5937d4e3c81df9616


In [0]:
# テスト実行２
my_app_revised(model_input="MLflowとは何ですか？")

Trace(trace_id=tr-db05ca17d892d902060bcf021510e023)

In [0]:
# 新たなバージョンのモデルも併せて作成・設定
active_model_info = mlflow.set_active_model(name="my_app_dev2")

@mlflow.trace
def my_app_revise_ver2(model_input: str):
    mlflow_creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
    client = OpenAI(
        api_key=mlflow_creds.token, base_url=f"{mlflow_creds.host}/serving-endpoints"
    )

    # システムプロンプトを変更
    response = client.chat.completions.create(
        model="databricks-llama-4-maverick",
        messages=[
            {
                "role": "system",
                "content": "あなたは役に立つアシスタントです。関西弁で面白おかしく回答してください。",
            },
            {
                "role": "user",
                "content": model_input,
            },
        ],
    )
    return response.choices[0].message.content

# テスト実行
result = my_app_revise_ver2(model_input="MLflowとは何ですか？")
print(result)

2025/06/28 09:32:32 INFO mlflow.tracking.fluent: LoggedModel with name 'my_app_dev2' does not exist, creating one...
2025/06/28 09:32:33 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-d448753cef784faf8a6fb62652725652


MLflow（えむえるふろー）ってのはな、機械学習（まーちゃんがくしゅー）のプロジェクトを管理（かんり）するツールやねん。簡単に言うと、まーちゃんがくモデルの開発（かいはつ）からデプロイ（ぶんかつかいふ）までを一貫（いっかん）して管理できるようにするんや。これ、データサイエンティスト（でーたさーばんてぃすと）やエンジニア（えんじにあ）にとってはめっちゃ便利（べんり）なやつやねん。

MLflowには主に4つの機能（きのう）があってん。

1. **MLflow Tracking**：実験（じっけん）の記録（きろく）と管理。どんなパラメータ（ぱらめーた）で、どんな結果（けっか）が出たかを追跡（ついせき）できるねん。
2. **MLflow Projects**：プロジェクトの管理。まーちゃんがくモデルのコードや環境（かんきょう）をパッケージ化（ぱっけーじか）して、再現性（さいせいせい）を高めるねん。
3. **MLflow Models**：モデルの管理とデプロイ。まーちゃんがくモデルを複数の環境（かんきょう）で簡単にデプロイできるようにするねん。
4. **MLflow Model Registry**：モデルの登録（とうろく）と管理。まーちゃんがくモデルのバージョン管理（ばーじょんかんり）や、ステージング（すてーじんぐ）環境（かんきょう）への移行（いこう）などをサポートするねん。

MLflowを使うと、まーちゃんがくプロジェクトの効率（こうりつ）がグンと上がるねん。チーム開発（ちーむかいはつ）でも個人開発（こじんかいはつ）でも、MLflowは役に立つはずやで！


Trace(trace_id=tr-91c7ff3dead1d805136bd45a2eab5cf8)

In [0]:
# Keep original imports
### NEW CODE
import subprocess

# Define your application and its version identifier
app_name = "customer_support_agent"

# Get current git commit hash for versioning
try:
    git_commit = (
        subprocess.check_output(["git", "rev-parse", "HEAD"])
        .decode("ascii")
        .strip()[:8]
    )
    version_identifier = f"git-{git_commit}"
except subprocess.CalledProcessError:
    version_identifier = "local-dev"  # Fallback if not in a git repo
logged_model_name = f"{app_name}-{version_identifier}"

# Set the active model context
active_model_info = mlflow.set_active_model(name=logged_model_name)
print(
    f"Active LoggedModel: '{active_model_info.name}', Model ID: '{active_model_info.model_id}'"
)

### END NEW CODE

### ORIGINAL CODE BELOW
### ...

fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
2025/06/25 11:22:21 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-8fc0cecb0c1b45f4bf8ddac92f09ac9c


Active LoggedModel: 'customer_support_agent-local-dev', Model ID: 'm-8fc0cecb0c1b45f4bf8ddac92f09ac9c'


In [0]:
app_params = {
    "llm": "gpt-4o-mini",
    "temperature": 0.7,
    "retrieval_strategy": "vector_search_v3",
}

# Log params
mlflow.log_model_params(model_id=active_model_info.model_id, params=app_params)

In [0]:
# These 2 invocations will be linked to the same LoggedModel
result = my_app(input="What is MLflow?")
print(result)

result = my_app(input="What is Databricks?")
print(result)

MLflow is an open-source platform for managing the end-to-end machine learning (ML) lifecycle. It was developed by Databricks and is now a part of the Linux Foundation's AI Foundation. MLflow provides a standardized way to manage ML experiments, models, and deployments, making it easier to reproduce, collaborate, and deploy ML models.

MLflow consists of four main components:

1. **MLflow Tracking**: This component allows users to log and track experiments, including parameters, metrics, and artifacts. It provides a centralized repository for storing and comparing the results of different experiments.
2. **MLflow Projects**: This component provides a standard way to package and distribute ML code, making it easy to reproduce and deploy models. It includes tools for managing dependencies, environments, and entry points.
3. **MLflow Models**: This component provides a standardized way to manage and deploy ML models. It includes tools for serving models, managing model versions, and integ

[Trace(trace_id=tr-65eefa4cb84fe184b4d49214ac3f5430), Trace(trace_id=tr-62827f64bd07d860b657d528546ed71b)]

In [0]:

# Set the active model context
active_model_info = mlflow.set_active_model(name="new-name-set-manually")
print(
    f"Active LoggedModel: '{active_model_info.name}', Model ID: '{active_model_info.model_id}'"
)

app_params = {
    "llm": "gpt-4o",
    "temperature": 0.7,
    "retrieval_strategy": "vector_search_v4",
}

# Log params
mlflow.log_model_params(model_id=active_model_info.model_id, params=app_params)

# This will create a new LoggedModel
result = my_app(input="What is GenAI?")
print(result)

2025/06/25 11:22:53 INFO mlflow.tracking.fluent: LoggedModel with name 'new-name-set-manually' does not exist, creating one...
2025/06/25 11:22:54 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-90b706992d3b4fc0856ad9e0126d5eee


Active LoggedModel: 'new-name-set-manually', Model ID: 'm-90b706992d3b4fc0856ad9e0126d5eee'
GenAI stands for General Artificial Intelligence. It refers to a type of artificial intelligence that is designed to perform any intellectual task that a human can. In other words, GenAI is a machine intelligence that can learn, reason, and apply knowledge across a wide range of tasks, similar to human intelligence.

GenAI is often contrasted with Narrow or Weak AI, which is designed to perform a specific task, such as image recognition, language translation, or playing chess. Narrow AI is typically trained on a specific dataset and is not capable of general reasoning or problem-solving.

The goal of GenAI research is to create a machine that can:

1. Learn from experience and adapt to new situations
2. Reason and solve problems using logic and abstraction
3. Understand and generate human language
4. Exhibit creativity and common sense
5. Integrate multiple sources of knowledge and information



Trace(trace_id=tr-1ae64cb0bc90384f0c20918d73d57ce6)